In [1]:
#import and setup
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import random
from api_key import api_key

#import finnhub
import finnhub
finnhub_client = finnhub.Client(api_key=api_key)

#finnhub utcdate codes for slicing by year for each stock
#2019 times
beg_2019 = 1546300800
end_2019 = 1577836799
#2020 times
beg_2020 = 1577836800
end_2020 = 1609459199
#2021 times
beg_2021 = 1609459200
end_2021 = 1640995199

In [4]:
clean_company_industry = pd.read_csv('output/clean_company_industry.csv')
### DO NOT RUN ###
# Randomly sample all US Large Cap companies in a given industry for only 2 companies per industry
ten_stock_df = pd.DataFrame()
for industry in list(clean_company_industry.Industry.unique()):
    df_temp = clean_company_industry.loc[clean_company_industry['Industry']==industry]
    if len(df_temp) > 2:
        random_sample = random.sample(list(df_temp['Ticker']),k=3)
        df_temp = df_temp.loc[df_temp['Ticker'].isin(random_sample)]
        ten_stock_df = ten_stock_df.append(df_temp)
ten_stock_df

C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_17552\2143931630.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ten_stock_df = ten_stock_df.append(df_temp)
C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_17552\2143931630.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ten_stock_df = ten_stock_df.append(df_temp)
C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_17552\2143931630.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ten_stock_df = ten_stock_df.append(df_temp)
C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_17552\2143931630.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ten_stoc

,Unnamed: 0,Ticker,Company Name,Industry,Country,Market Cap
44,967,DDOG,Datadog Inc,Technology,US,30676.488212
223,4096,BILL,Bill.com Holdings Inc,Technology,US,15688.310199
326,4710,FICO,Fair Isaac Corp,Technology,US,11105.550741
180,3777,ABC,Amerisourcebergen Corp,Health Care,US,30614.092181
236,4210,CAH,Cardinal Health Inc,Health Care,US,19345.654493
...,...,...,...,...,...,...
266,4375,CSL,Carlisle Companies Inc,Building,US,15308.318988
548,6363,WMS,Advanced Drainage Systems Inc,Building,US,11153.408965
251,4302,CLX,Clorox Co,Consumer products,US,17593.841731
283,4457,DHI,D R Horton Inc,Consumer products,US,24803.230560


In [16]:
# Two random stocks from industries we are interested in observing
ten_stock_industry_df = pd.DataFrame()
ten_industry_list = ['Airlines', 'Automobiles', 'Consumer products', 'Health Care', 'Hotels, Restaurants & Leisure', 'Logistics & Transportation', 'Pharmaceuticals', 'Real Estate', 'Telecommunication', 'Technology']
for industry in ten_industry_list:
    ten_stock_industry_df = ten_stock_industry_df.append(ten_stock_df.loc[ten_stock_df['Industry']==industry])
ten_stock_industry_df = ten_stock_industry_df.drop(columns='Unnamed: 0')
        
# Save the DataFrame to a .csv file
ten_stock_industry_df = ten_stock_industry_df.reset_index()
ten_stock_industry_df = ten_stock_industry_df.drop(columns='index')
ten_stock_industry_df.to_csv('output/ten_stock_industry_df.csv', index=False)
ten_stock_industry_df

C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_17552\1506282729.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ten_stock_industry_df = ten_stock_industry_df.append(ten_stock_df.loc[ten_stock_df['Industry']==industry])
C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_17552\1506282729.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ten_stock_industry_df = ten_stock_industry_df.append(ten_stock_df.loc[ten_stock_df['Industry']==industry])
C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_17552\1506282729.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ten_stock_industry_df = ten_stock_industry_df.append(ten_stock_df.loc[ten_stock_df['Industry']==industry])
C:\Users\dejans.THOMASPUMP\AppData\Lo

,Ticker,Company Name,Industry,Country,Market Cap
0,UAL,United Airlines Holdings Inc,Airlines,US,11537.355102
1,DAL,Delta Air Lines Inc,Airlines,US,19838.664269
2,LUV,Southwest Airlines Co,Airlines,US,21799.685466
3,LCID,Lucid Group Inc,Automobiles,US,25359.750711
4,TSLA,Tesla Inc,Automobiles,US,846694.931971
5,GM,General Motors Co,Automobiles,US,56091.145112
6,CLX,Clorox Co,Consumer products,US,17593.841731
7,DHI,D R Horton Inc,Consumer products,US,24803.230560
8,PG,Procter & Gamble Co,Consumer products,US,327751.219046
9,ABC,Amerisourcebergen Corp,Health Care,US,30614.092181


In [53]:
#def function to create a dataframe for each stock
def stock_df(stock_name, beg_year, end_year):
    df_name = pd.DataFrame(finnhub_client.stock_candles(stock_name, 'W', beg_year, end_year)) 
    df_name = df_name.drop(['s'], axis=1)
    rename = {'c':'Close','h': 'High', 'l': 'Low', 'o': 'Open', 't': 'Time', 'v': 'Volume'}
    df_name.rename(columns=rename, inplace=True)
    df_name['Time'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') for x in df_name['Time']]
    return df_name



### Pulling in stock price for all tickers in DataFrame

In [58]:
UAL_2019_df = stock_df('UAL', beg_2019, end_2019)
UAL_2019_df = stock_df('UAL', beg_2019, end_2019)
UAL_2019_df = stock_df('UAL', beg_2019, end_2019)

UAL_2020_df = stock_df('UAL', beg_2020, end_2020)
UAL_2020_df = stock_df('UAL', beg_2020, end_2020)
UAL_2020_df = stock_df('UAL', beg_2020, end_2020)

UAL_2021_df = stock_df('UAL', beg_2021, end_2021)
UAL_2021_df = stock_df('UAL', beg_2021, end_2021)
UAL_2021_df = stock_df('UAL', beg_2021, end_2021)

In [59]:
DAL_2019_df = stock_df('DAL', beg_2019, end_2019)
DAL_2019_df = stock_df('DAL', beg_2019, end_2019)
DAL_2019_df = stock_df('DAL', beg_2019, end_2019)

DAL_2020_df = stock_df('DAL', beg_2020, end_2020)
DAL_2020_df = stock_df('DAL', beg_2020, end_2020)
DAL_2020_df = stock_df('DAL', beg_2020, end_2020)

DAL_2021_df = stock_df('DAL', beg_2021, end_2021)
DAL_2021_df = stock_df('DAL', beg_2021, end_2021)
DAL_2021_df = stock_df('DAL', beg_2021, end_2021)

In [60]:
LUV_2019_df = stock_df('LUV', beg_2019, end_2019)
LUV_2019_df = stock_df('LUV', beg_2019, end_2019)
LUV_2019_df = stock_df('LUV', beg_2019, end_2019)

LUV_2020_df = stock_df('LUV', beg_2020, end_2020)
LUV_2020_df = stock_df('LUV', beg_2020, end_2020)
LUV_2020_df = stock_df('LUV', beg_2020, end_2020)

LUV_2021_df = stock_df('LUV', beg_2021, end_2021)
LUV_2021_df = stock_df('LUV', beg_2021, end_2021)
LUV_2021_df = stock_df('LUV', beg_2021, end_2021)

In [62]:
F_2019_df = stock_df('F', beg_2019, end_2019)
F_2019_df = stock_df('F', beg_2019, end_2019)
F_2019_df = stock_df('F', beg_2019, end_2019)

F_2020_df = stock_df('F', beg_2020, end_2020)
F_2020_df = stock_df('F', beg_2020, end_2020)
F_2020_df = stock_df('F', beg_2020, end_2020)

F_2021_df = stock_df('F', beg_2021, end_2021)
F_2021_df = stock_df('F', beg_2021, end_2021)
F_2021_df = stock_df('F', beg_2021, end_2021)

In [63]:
TSLA_2019_df = stock_df('TSLA', beg_2019, end_2019)
TSLA_2019_df = stock_df('TSLA', beg_2019, end_2019)
TSLA_2019_df = stock_df('TSLA', beg_2019, end_2019)

TSLA_2020_df = stock_df('TSLA', beg_2020, end_2020)
TSLA_2020_df = stock_df('TSLA', beg_2020, end_2020)
TSLA_2020_df = stock_df('TSLA', beg_2020, end_2020)

TSLA_2021_df = stock_df('TSLA', beg_2021, end_2021)
TSLA_2021_df = stock_df('TSLA', beg_2021, end_2021)
TSLA_2021_df = stock_df('TSLA', beg_2021, end_2021)

In [64]:
GM_2019_df = stock_df('GM', beg_2019, end_2019)
GM_2019_df = stock_df('GM', beg_2019, end_2019)
GM_2019_df = stock_df('GM', beg_2019, end_2019)

GM_2020_df = stock_df('GM', beg_2020, end_2020)
GM_2020_df = stock_df('GM', beg_2020, end_2020)
GM_2020_df = stock_df('GM', beg_2020, end_2020)

GM_2021_df = stock_df('GM', beg_2021, end_2021)
GM_2021_df = stock_df('GM', beg_2021, end_2021)
GM_2021_df = stock_df('GM', beg_2021, end_2021)

In [65]:
CLX_2019_df = stock_df('CLX', beg_2019, end_2019)
CLX_2019_df = stock_df('CLX', beg_2019, end_2019)
CLX_2019_df = stock_df('CLX', beg_2019, end_2019)

CLX_2020_df = stock_df('CLX', beg_2020, end_2020)
CLX_2020_df = stock_df('CLX', beg_2020, end_2020)
CLX_2020_df = stock_df('CLX', beg_2020, end_2020)

CLX_2021_df = stock_df('CLX', beg_2021, end_2021)
CLX_2021_df = stock_df('CLX', beg_2021, end_2021)
CLX_2021_df = stock_df('CLX', beg_2021, end_2021)

In [66]:
DHI_2019_df = stock_df('DHI', beg_2019, end_2019)
DHI_2019_df = stock_df('DHI', beg_2019, end_2019)
DHI_2019_df = stock_df('DHI', beg_2019, end_2019)

DHI_2020_df = stock_df('DHI', beg_2020, end_2020)
DHI_2020_df = stock_df('DHI', beg_2020, end_2020)
DHI_2020_df = stock_df('DHI', beg_2020, end_2020)

DHI_2021_df = stock_df('DHI', beg_2021, end_2021)
DHI_2021_df = stock_df('DHI', beg_2021, end_2021)
DHI_2021_df = stock_df('DHI', beg_2021, end_2021)

In [67]:
PG_2019_df = stock_df('PG', beg_2019, end_2019)
PG_2019_df = stock_df('PG', beg_2019, end_2019)
PG_2019_df = stock_df('PG', beg_2019, end_2019)

PG_2020_df = stock_df('PG', beg_2020, end_2020)
PG_2020_df = stock_df('PG', beg_2020, end_2020)
PG_2020_df = stock_df('PG', beg_2020, end_2020)

PG_2021_df = stock_df('PG', beg_2021, end_2021)
PG_2021_df = stock_df('PG', beg_2021, end_2021)
PG_2021_df = stock_df('PG', beg_2021, end_2021)

In [68]:
ABC_2019_df = stock_df('ABC', beg_2019, end_2019)
ABC_2019_df = stock_df('ABC', beg_2019, end_2019)
ABC_2019_df = stock_df('ABC', beg_2019, end_2019)

ABC_2020_df = stock_df('ABC', beg_2020, end_2020)
ABC_2020_df = stock_df('ABC', beg_2020, end_2020)
ABC_2020_df = stock_df('ABC', beg_2020, end_2020)

ABC_2021_df = stock_df('ABC', beg_2021, end_2021)
ABC_2021_df = stock_df('ABC', beg_2021, end_2021)
ABC_2021_df = stock_df('ABC', beg_2021, end_2021)

In [69]:
CAH_2019_df = stock_df('CAH', beg_2019, end_2019)
CAH_2019_df = stock_df('CAH', beg_2019, end_2019)
CAH_2019_df = stock_df('CAH', beg_2019, end_2019)

CAH_2020_df = stock_df('CAH', beg_2020, end_2020)
CAH_2020_df = stock_df('CAH', beg_2020, end_2020)
CAH_2020_df = stock_df('CAH', beg_2020, end_2020)

CAH_2021_df = stock_df('CAH', beg_2021, end_2021)
CAH_2021_df = stock_df('CAH', beg_2021, end_2021)
CAH_2021_df = stock_df('CAH', beg_2021, end_2021)

In [70]:
COO_2019_df = stock_df('COO', beg_2019, end_2019)
COO_2019_df = stock_df('COO', beg_2019, end_2019)
COO_2019_df = stock_df('COO', beg_2019, end_2019)

COO_2020_df = stock_df('COO', beg_2020, end_2020)
COO_2020_df = stock_df('COO', beg_2020, end_2020)
COO_2020_df = stock_df('COO', beg_2020, end_2020)

COO_2021_df = stock_df('COO', beg_2021, end_2021)
COO_2021_df = stock_df('COO', beg_2021, end_2021)
COO_2021_df = stock_df('COO', beg_2021, end_2021)

In [72]:
DPZ_2019_df = stock_df('DPZ', beg_2019, end_2019)
DPZ_2019_df = stock_df('DPZ', beg_2019, end_2019)
DPZ_2019_df = stock_df('DPZ', beg_2019, end_2019)

DPZ_2020_df = stock_df('DPZ', beg_2020, end_2020)
DPZ_2020_df = stock_df('DPZ', beg_2020, end_2020)
DPZ_2020_df = stock_df('DPZ', beg_2020, end_2020)

DPZ_2021_df = stock_df('DPZ', beg_2021, end_2021)
DPZ_2021_df = stock_df('DPZ', beg_2021, end_2021)
DPZ_2021_df = stock_df('DPZ', beg_2021, end_2021)

In [73]:
LVS_2019_df = stock_df('LVS', beg_2019, end_2019)
LVS_2019_df = stock_df('LVS', beg_2019, end_2019)
LVS_2019_df = stock_df('LVS', beg_2019, end_2019)

LVS_2020_df = stock_df('LVS', beg_2020, end_2020)
LVS_2020_df = stock_df('LVS', beg_2020, end_2020)
LVS_2020_df = stock_df('LVS', beg_2020, end_2020)

LVS_2021_df = stock_df('LVS', beg_2021, end_2021)
LVS_2021_df = stock_df('LVS', beg_2021, end_2021)
LVS_2021_df = stock_df('LVS', beg_2021, end_2021)

In [74]:
MCD_2019_df = stock_df('MCD', beg_2019, end_2019)
MCD_2019_df = stock_df('MCD', beg_2019, end_2019)
MCD_2019_df = stock_df('MCD', beg_2019, end_2019)

MCD_2020_df = stock_df('MCD', beg_2020, end_2020)
MCD_2020_df = stock_df('MCD', beg_2020, end_2020)
MCD_2020_df = stock_df('MCD', beg_2020, end_2020)

MCD_2021_df = stock_df('MCD', beg_2021, end_2021)
MCD_2021_df = stock_df('MCD', beg_2021, end_2021)
MCD_2021_df = stock_df('MCD', beg_2021, end_2021)

In [75]:
CHRW_2019_df = stock_df('CHRW', beg_2019, end_2019)
CHRW_2019_df = stock_df('CHRW', beg_2019, end_2019)
CHRW_2019_df = stock_df('CHRW', beg_2019, end_2019)

CHRW_2020_df = stock_df('CHRW', beg_2020, end_2020)
CHRW_2020_df = stock_df('CHRW', beg_2020, end_2020)
CHRW_2020_df = stock_df('CHRW', beg_2020, end_2020)

CHRW_2021_df = stock_df('CHRW', beg_2021, end_2021)
CHRW_2021_df = stock_df('CHRW', beg_2021, end_2021)
CHRW_2021_df = stock_df('CHRW', beg_2021, end_2021)

In [76]:
EXPD_2019_df = stock_df('EXPD', beg_2019, end_2019)
EXPD_2019_df = stock_df('EXPD', beg_2019, end_2019)
EXPD_2019_df = stock_df('EXPD', beg_2019, end_2019)

EXPD_2020_df = stock_df('EXPD', beg_2020, end_2020)
EXPD_2020_df = stock_df('EXPD', beg_2020, end_2020)
EXPD_2020_df = stock_df('EXPD', beg_2020, end_2020)

EXPD_2021_df = stock_df('EXPD', beg_2021, end_2021)
EXPD_2021_df = stock_df('EXPD', beg_2021, end_2021)
EXPD_2021_df = stock_df('EXPD', beg_2021, end_2021)

In [77]:
FDX_2019_df = stock_df('FDX', beg_2019, end_2019)
FDX_2019_df = stock_df('FDX', beg_2019, end_2019)
FDX_2019_df = stock_df('FDX', beg_2019, end_2019)

FDX_2020_df = stock_df('FDX', beg_2020, end_2020)
FDX_2020_df = stock_df('FDX', beg_2020, end_2020)
FDX_2020_df = stock_df('FDX', beg_2020, end_2020)

FDX_2021_df = stock_df('FDX', beg_2021, end_2021)
FDX_2021_df = stock_df('FDX', beg_2021, end_2021)
FDX_2021_df = stock_df('FDX', beg_2021, end_2021)

In [78]:
BMY_2019_df = stock_df('BMY', beg_2019, end_2019)
BMY_2019_df = stock_df('BMY', beg_2019, end_2019)
BMY_2019_df = stock_df('BMY', beg_2019, end_2019)

BMY_2020_df = stock_df('BMY', beg_2020, end_2020)
BMY_2020_df = stock_df('BMY', beg_2020, end_2020)
BMY_2020_df = stock_df('BMY', beg_2020, end_2020)

BMY_2021_df = stock_df('BMY', beg_2021, end_2021)
BMY_2021_df = stock_df('BMY', beg_2021, end_2021)
BMY_2021_df = stock_df('BMY', beg_2021, end_2021)

In [79]:
LLY_2019_df = stock_df('LLY', beg_2019, end_2019)
LLY_2019_df = stock_df('LLY', beg_2019, end_2019)
LLY_2019_df = stock_df('LLY', beg_2019, end_2019)

LLY_2020_df = stock_df('LLY', beg_2020, end_2020)
LLY_2020_df = stock_df('LLY', beg_2020, end_2020)
LLY_2020_df = stock_df('LLY', beg_2020, end_2020)

LLY_2021_df = stock_df('LLY', beg_2021, end_2021)
LLY_2021_df = stock_df('LLY', beg_2021, end_2021)
LLY_2021_df = stock_df('LLY', beg_2021, end_2021)

In [80]:
ZTS_2019_df = stock_df('ZTS', beg_2019, end_2019)
ZTS_2019_df = stock_df('ZTS', beg_2019, end_2019)
ZTS_2019_df = stock_df('ZTS', beg_2019, end_2019)

ZTS_2020_df = stock_df('ZTS', beg_2020, end_2020)
ZTS_2020_df = stock_df('ZTS', beg_2020, end_2020)
ZTS_2020_df = stock_df('ZTS', beg_2020, end_2020)

ZTS_2021_df = stock_df('ZTS', beg_2021, end_2021)
ZTS_2021_df = stock_df('ZTS', beg_2021, end_2021)
ZTS_2021_df = stock_df('ZTS', beg_2021, end_2021)

In [81]:
ARE_2019_df = stock_df('ARE', beg_2019, end_2019)
ARE_2019_df = stock_df('ARE', beg_2019, end_2019)
ARE_2019_df = stock_df('ARE', beg_2019, end_2019)

ARE_2020_df = stock_df('ARE', beg_2020, end_2020)
ARE_2020_df = stock_df('ARE', beg_2020, end_2020)
ARE_2020_df = stock_df('ARE', beg_2020, end_2020)

ARE_2021_df = stock_df('ARE', beg_2021, end_2021)
ARE_2021_df = stock_df('ARE', beg_2021, end_2021)
ARE_2021_df = stock_df('ARE', beg_2021, end_2021)

In [82]:
CUBE_2019_df = stock_df('CUBE', beg_2019, end_2019)
CUBE_2019_df = stock_df('CUBE', beg_2019, end_2019)
CUBE_2019_df = stock_df('CUBE', beg_2019, end_2019)

CUBE_2020_df = stock_df('CUBE', beg_2020, end_2020)
CUBE_2020_df = stock_df('CUBE', beg_2020, end_2020)
CUBE_2020_df = stock_df('CUBE', beg_2020, end_2020)

CUBE_2021_df = stock_df('CUBE', beg_2021, end_2021)
CUBE_2021_df = stock_df('CUBE', beg_2021, end_2021)
CUBE_2021_df = stock_df('CUBE', beg_2021, end_2021)

In [83]:
DRE_2019_df = stock_df('DRE', beg_2019, end_2019)
DRE_2019_df = stock_df('DRE', beg_2019, end_2019)
DRE_2019_df = stock_df('DRE', beg_2019, end_2019)

DRE_2020_df = stock_df('DRE', beg_2020, end_2020)
DRE_2020_df = stock_df('DRE', beg_2020, end_2020)
DRE_2020_df = stock_df('DRE', beg_2020, end_2020)

DRE_2021_df = stock_df('DRE', beg_2021, end_2021)
DRE_2021_df = stock_df('DRE', beg_2021, end_2021)
DRE_2021_df = stock_df('DRE', beg_2021, end_2021)

In [84]:
TMUS_2019_df = stock_df('TMUS', beg_2019, end_2019)
TMUS_2019_df = stock_df('TMUS', beg_2019, end_2019)
TMUS_2019_df = stock_df('TMUS', beg_2019, end_2019)

TMUS_2020_df = stock_df('TMUS', beg_2020, end_2020)
TMUS_2020_df = stock_df('TMUS', beg_2020, end_2020)
TMUS_2020_df = stock_df('TMUS', beg_2020, end_2020)

TMUS_2021_df = stock_df('TMUS', beg_2021, end_2021)
TMUS_2021_df = stock_df('TMUS', beg_2021, end_2021)
TMUS_2021_df = stock_df('TMUS', beg_2021, end_2021)

In [85]:
LUMN_2019_df = stock_df('LUMN', beg_2019, end_2019)
LUMN_2019_df = stock_df('LUMN', beg_2019, end_2019)
LUMN_2019_df = stock_df('LUMN', beg_2019, end_2019)

LUMN_2020_df = stock_df('LUMN', beg_2020, end_2020)
LUMN_2020_df = stock_df('LUMN', beg_2020, end_2020)
LUMN_2020_df = stock_df('LUMN', beg_2020, end_2020)

LUMN_2021_df = stock_df('LUMN', beg_2021, end_2021)
LUMN_2021_df = stock_df('LUMN', beg_2021, end_2021)
LUMN_2021_df = stock_df('LUMN', beg_2021, end_2021)

In [86]:
T_2019_df = stock_df('T', beg_2019, end_2019)
T_2019_df = stock_df('T', beg_2019, end_2019)
T_2019_df = stock_df('T', beg_2019, end_2019)

T_2020_df = stock_df('T', beg_2020, end_2020)
T_2020_df = stock_df('T', beg_2020, end_2020)
T_2020_df = stock_df('T', beg_2020, end_2020)

T_2021_df = stock_df('T', beg_2021, end_2021)
T_2021_df = stock_df('T', beg_2021, end_2021)
T_2021_df = stock_df('T', beg_2021, end_2021)

In [136]:
META_2019_df = stock_df('META', beg_2019, end_2019)
META_2019_df = stock_df('META', beg_2019, end_2019)
META_2019_df = stock_df('META', beg_2019, end_2019)

META_2020_df = stock_df('META', beg_2020, end_2020)
META_2020_df = stock_df('META', beg_2020, end_2020)
META_2020_df = stock_df('META', beg_2020, end_2020)

META_2021_df = stock_df('META', beg_2021, end_2021)
META_2021_df = stock_df('META', beg_2021, end_2021)
META_2021_df = stock_df('META', beg_2021, end_2021)

In [137]:
AAPL_2019_df = stock_df('AAPL', beg_2019, end_2019)
AAPL_2019_df = stock_df('AAPL', beg_2019, end_2019)
AAPL_2019_df = stock_df('AAPL', beg_2019, end_2019)

AAPL_2020_df = stock_df('AAPL', beg_2020, end_2020)
AAPL_2020_df = stock_df('AAPL', beg_2020, end_2020)
AAPL_2020_df = stock_df('AAPL', beg_2020, end_2020)

AAPL_2021_df = stock_df('AAPL', beg_2021, end_2021)
AAPL_2021_df = stock_df('AAPL', beg_2021, end_2021)
AAPL_2021_df = stock_df('AAPL', beg_2021, end_2021)

In [138]:
GOOG_2019_df = stock_df('GOOG', beg_2019, end_2019)
GOOG_2019_df = stock_df('GOOG', beg_2019, end_2019)
GOOG_2019_df = stock_df('GOOG', beg_2019, end_2019)

GOOG_2020_df = stock_df('GOOG', beg_2020, end_2020)
GOOG_2020_df = stock_df('GOOG', beg_2020, end_2020)
GOOG_2020_df = stock_df('GOOG', beg_2020, end_2020)

GOOG_2021_df = stock_df('GOOG', beg_2021, end_2021)
GOOG_2021_df = stock_df('GOOG', beg_2021, end_2021)
GOOG_2021_df = stock_df('GOOG', beg_2021, end_2021)

### Calculating performance of each industry

In [115]:
airlines_2019_df = UAL_2019_df + DAL_2019_df + LUV_2019_df
airlines_close_2019 = airlines_2019_df['Close']/3
airlines_close_perc_2019 = round(((airlines_close_2019[52]-airlines_close_2019[0])/airlines_close_2019[0])*100,ndigits=2)
print(airlines_close_perc_2019)

airlines_2020_df = UAL_2020_df + DAL_2020_df + LUV_2020_df
airlines_close_2020 = airlines_2020_df['Close']/3
airlines_close_perc_2020 = round(((airlines_close_2020[52]-airlines_close_2020[0])/airlines_close_2020[0])*100,ndigits=2)
print(airlines_close_perc_2020)

airlines_2021_df = UAL_2021_df + DAL_2021_df + LUV_2021_df
airlines_close_2021 = airlines_2021_df['Close']/3
airlines_close_perc_2021 = round(((airlines_close_2021[51]-airlines_close_2021[0])/airlines_close_2021[0])*100,ndigits=2)
print(airlines_close_perc_2021)

12.54
-34.29
-3.38


In [122]:
automobiles_2019_df = F_2019_df + TSLA_2019_df + GM_2019_df
automobiles_close_2019 = automobiles_2019_df['Close']/3
automobiles_close_perc_2019 = round(((automobiles_close_2019[52]-automobiles_close_2019[0])/automobiles_close_2019[0])*100,ndigits=2)
print(automobiles_close_perc_2019)

automobiles_2020_df = F_2020_df + TSLA_2020_df + GM_2020_df
automobiles_close_2020 = automobiles_2020_df['Close']/3
automobiles_close_perc_2020 = round(((automobiles_close_2020[52]-automobiles_close_2020[0])/automobiles_close_2020[0])*100,ndigits=2)
print(automobiles_close_perc_2020)

automobiles_2021_df = F_2021_df + TSLA_2021_df + GM_2021_df
automobiles_close_2021 = automobiles_2021_df['Close']/3
automobiles_close_perc_2021 = round(((automobiles_close_2021[51]-automobiles_close_2021[0])/automobiles_close_2021[0])*100,ndigits=2)
print(automobiles_close_perc_2021)

17.17
276.07
26.05


In [123]:
consumer_products_2019_df = CLX_2019_df + DHI_2019_df + PG_2019_df
consumer_products_close_2019 = consumer_products_2019_df['Close']/3
consumer_products_close_perc_2019 = round(((consumer_products_close_2019[52]-consumer_products_close_2019[0])/consumer_products_close_2019[0])*100,ndigits=2)
print(consumer_products_close_perc_2019)

consumer_products_2020_df = CLX_2020_df + DHI_2020_df + PG_2020_df
consumer_products_close_2020 = consumer_products_2020_df['Close']/3
consumer_products_close_perc_2020 = round(((consumer_products_close_2020[52]-consumer_products_close_2020[0])/consumer_products_close_2020[0])*100,ndigits=2)
print(consumer_products_close_perc_2020)

consumer_products_2021_df = CLX_2021_df + DHI_2021_df + PG_2021_df
consumer_products_close_2021 = consumer_products_2021_df['Close']/3
consumer_products_close_perc_2021 = round(((consumer_products_close_2021[51]-consumer_products_close_2021[0])/consumer_products_close_2021[0])*100,ndigits=2)
print(consumer_products_close_perc_2021)

16.86
24.5
9.67


In [124]:
health_care_2019_df = ABC_2019_df + CAH_2019_df + COO_2019_df
health_care_close_2019 = health_care_2019_df['Close']/3
health_care_close_perc_2019 = round(((health_care_close_2019[52]-health_care_close_2019[0])/health_care_close_2019[0])*100,ndigits=2)
print(health_care_close_perc_2019)

health_care_2020_df = ABC_2020_df + CAH_2020_df + COO_2020_df
health_care_close_2020 = health_care_2020_df['Close']/3
health_care_close_perc_2020 = round(((health_care_close_2020[52]-health_care_close_2020[0])/health_care_close_2020[0])*100,ndigits=2)
print(health_care_close_perc_2020)

health_care_2021_df = ABC_2021_df + CAH_2021_df + COO_2021_df
health_care_close_2021 = health_care_2021_df['Close']/3
health_care_close_perc_2021 = round(((health_care_close_2021[51]-health_care_close_2021[0])/health_care_close_2021[0])*100,ndigits=2)
print(health_care_close_perc_2021)

24.23
11.24
15.27


In [125]:
hotels_restaurants_leisure_2019_df = DPZ_2019_df + LVS_2019_df + MCD_2019_df
hotels_restaurants_leisure_close_2019 = hotels_restaurants_leisure_2019_df['Close']/3
hotels_restaurants_leisure_close_perc_2019 = round(((hotels_restaurants_leisure_close_2019[52]-hotels_restaurants_leisure_close_2019[0])/hotels_restaurants_leisure_close_2019[0])*100,ndigits=2)
print(hotels_restaurants_leisure_close_perc_2019)

hotels_restaurants_leisure_2020_df = DPZ_2020_df + LVS_2020_df + MCD_2020_df
hotels_restaurants_leisure_close_2020 = hotels_restaurants_leisure_2020_df['Close']/3
hotels_restaurants_leisure_close_perc_2020 = round(((hotels_restaurants_leisure_close_2020[52]-hotels_restaurants_leisure_close_2020[0])/hotels_restaurants_leisure_close_2020[0])*100,ndigits=2)
print(hotels_restaurants_leisure_close_perc_2020)

hotels_restaurants_leisure_2021_df = DPZ_2021_df + LVS_2021_df + MCD_2021_df
hotels_restaurants_leisure_close_2021 = hotels_restaurants_leisure_2021_df['Close']/3
hotels_restaurants_leisure_close_perc_2021 = round(((hotels_restaurants_leisure_close_2021[51]-hotels_restaurants_leisure_close_2021[0])/hotels_restaurants_leisure_close_2021[0])*100,ndigits=2)
print(hotels_restaurants_leisure_close_perc_2021)

17.27
16.02
30.13


In [127]:
logistics_transportation_2019_df = CHRW_2019_df + FDX_2019_df + EXPD_2019_df
logistics_transportation_close_2019 = logistics_transportation_2019_df['Close']/3
logistics_transportation_close_perc_2019 = round(((logistics_transportation_close_2019[52]-logistics_transportation_close_2019[0])/logistics_transportation_close_2019[0])*100,ndigits=2)
print(logistics_transportation_close_perc_2019)

logistics_transportation_2020_df = CHRW_2020_df + FDX_2020_df + EXPD_2020_df
logistics_transportation_close_2020 = logistics_transportation_2020_df['Close']/3
logistics_transportation_close_perc_2020 = round(((logistics_transportation_close_2020[52]-logistics_transportation_close_2020[0])/logistics_transportation_close_2020[0])*100,ndigits=2)
print(logistics_transportation_close_perc_2020)

logistics_transportation_2021_df = CHRW_2021_df + FDX_2021_df + EXPD_2021_df
logistics_transportation_close_2021 = logistics_transportation_2021_df['Close']/3
logistics_transportation_close_perc_2021 = round(((logistics_transportation_close_2021[51]-logistics_transportation_close_2021[0])/logistics_transportation_close_2021[0])*100,ndigits=2)
print(logistics_transportation_close_perc_2021)

-2.67
44.95
14.27


In [128]:
pharmaceuticals_2019_df = BMY_2019_df + LLY_2019_df + ZTS_2019_df
pharmaceuticals_close_2019 = pharmaceuticals_2019_df['Close']/3
pharmaceuticals_close_perc_2019 = round(((pharmaceuticals_close_2019[52]-pharmaceuticals_close_2019[0])/pharmaceuticals_close_2019[0])*100,ndigits=2)
print(pharmaceuticals_close_perc_2019)

pharmaceuticals_2020_df = BMY_2020_df + LLY_2020_df + ZTS_2020_df
pharmaceuticals_close_2020 = pharmaceuticals_2020_df['Close']/3
pharmaceuticals_close_perc_2020 = round(((pharmaceuticals_close_2020[52]-pharmaceuticals_close_2020[0])/pharmaceuticals_close_2020[0])*100,ndigits=2)
print(pharmaceuticals_close_perc_2020)

pharmaceuticals_2021_df = BMY_2021_df + LLY_2021_df + ZTS_2021_df
pharmaceuticals_close_2021 = pharmaceuticals_2021_df['Close']/3
pharmaceuticals_close_perc_2021 = round(((pharmaceuticals_close_2021[51]-pharmaceuticals_close_2021[0])/pharmaceuticals_close_2021[0])*100,ndigits=2)
print(pharmaceuticals_close_perc_2021)

32.86
19.62
47.33


In [129]:
real_estate_2019_df = ARE_2019_df + CUBE_2019_df + DRE_2019_df
real_estate_close_2019 = real_estate_2019_df['Close']/3
real_estate_close_perc_2019 = round(((real_estate_close_2019[52]-real_estate_close_2019[0])/real_estate_close_2019[0])*100,ndigits=2)
print(real_estate_close_perc_2019)

real_estate_2020_df = ARE_2020_df + CUBE_2020_df + DRE_2020_df
real_estate_close_2020 = real_estate_2020_df['Close']/3
real_estate_close_perc_2020 = round(((real_estate_close_2020[52]-real_estate_close_2020[0])/real_estate_close_2020[0])*100,ndigits=2)
print(real_estate_close_perc_2020)

real_estate_2021_df = ARE_2021_df + CUBE_2021_df + DRE_2021_df
real_estate_close_2021 = real_estate_2021_df['Close']/3
real_estate_close_perc_2021 = round(((real_estate_close_2021[51]-real_estate_close_2021[0])/real_estate_close_2021[0])*100,ndigits=2)
print(real_estate_close_perc_2021)

34.99
9.24
44.87


In [130]:
telecommunication_2019_df = TMUS_2019_df + LUMN_2019_df + T_2019_df
telecommunication_close_2019 = telecommunication_2019_df['Close']/3
telecommunication_close_perc_2019 = round(((telecommunication_close_2019[52]-telecommunication_close_2019[0])/telecommunication_close_2019[0])*100,ndigits=2)
print(telecommunication_close_perc_2019)

telecommunication_2020_df = TMUS_2020_df + LUMN_2020_df + T_2020_df
telecommunication_close_2020 = telecommunication_2020_df['Close']/3
telecommunication_close_perc_2020 = round(((telecommunication_close_2020[52]-telecommunication_close_2020[0])/telecommunication_close_2020[0])*100,ndigits=2)
print(telecommunication_close_perc_2020)

telecommunication_2021_df = TMUS_2021_df + LUMN_2021_df + T_2021_df
telecommunication_close_2021 = telecommunication_2021_df['Close']/3
telecommunication_close_perc_2021 = round(((telecommunication_close_2021[51]-telecommunication_close_2021[0])/telecommunication_close_2021[0])*100,ndigits=2)
print(telecommunication_close_perc_2021)

14.45
32.1
-11.79


In [139]:
technology_2019_df = META_2019_df + AAPL_2019_df + GOOG_2019_df
technology_close_2019 = technology_2019_df['Close']/3
technology_close_perc_2019 = round(((technology_close_2019[52]-technology_close_2019[0])/technology_close_2019[0])*100,ndigits=2)
print(technology_close_perc_2019)

technology_2020_df = META_2020_df + AAPL_2020_df + GOOG_2020_df
technology_close_2020 = technology_2020_df['Close']/3
technology_close_perc_2020 = round(((technology_close_2020[52]-technology_close_2020[0])/technology_close_2020[0])*100,ndigits=2)
print(technology_close_perc_2020)

technology_2021_df = META_2021_df + AAPL_2021_df + GOOG_2021_df
technology_close_2021 = technology_2021_df['Close']/3
technology_close_perc_2021 = round(((technology_close_2021[51]-technology_close_2021[0])/technology_close_2021[0])*100,ndigits=2)
print(technology_close_perc_2021)

50.56
40.31
36.45


In [161]:
# Create a dictionary of percentage returns
percentage_return_dict = {
    'Airlines 2019 Performance': [airlines_close_perc_2019],
    'Airlines 2020 Performance': [airlines_close_perc_2020],
    'Airlines 2021 Performance': [airlines_close_perc_2021],
    'Automobiles 2019 Performance': [automobiles_close_perc_2019],
    'Automobiles 2020 Performance': [automobiles_close_perc_2020],
    'Automobiles 2021 Performance': [automobiles_close_perc_2021],
    'Consumer Products 2019 Performance': [consumer_products_close_perc_2019],
    'Consumer Products 2020 Performance': [consumer_products_close_perc_2020],
    'Consumer Products 2021 Performance': [consumer_products_close_perc_2021],
    'Health Care 2019 Performance': [health_care_close_perc_2019],
    'Health Care 2020 Performance': [health_care_close_perc_2020],
    'Health Care 2021 Performance': [health_care_close_perc_2021],
    'Hotels Restaurants and Leisure 2019 Performance': [hotels_restaurants_leisure_close_perc_2019],
    'Hotels Restaurants and Leisure 2020 Performance': [hotels_restaurants_leisure_close_perc_2020],
    'Hotels Restaurants and Leisure 2021 Performance': [hotels_restaurants_leisure_close_perc_2021],
    'Logistics and Transportation 2019 Performance': [logistics_transportation_close_perc_2019],
    'Logistics and Transportation 2020 Performance': [logistics_transportation_close_perc_2020],
    'Logistics and Transportation 2021 Performance': [logistics_transportation_close_perc_2021],
    'Pharmaceuticals 2019 Performance': [pharmaceuticals_close_perc_2019],
    'Pharmaceuticals 2020 Performance': [pharmaceuticals_close_perc_2020],
    'Pharmaceuticals 2021 Performance': [pharmaceuticals_close_perc_2021],
    'Real Estate 2019 Performance': [real_estate_close_perc_2019],
    'Real Estate 2020 Performance': [real_estate_close_perc_2020],
    'Real Estate 2021 Performance': [real_estate_close_perc_2021],
    'Telecommunication 2019 Performance': [telecommunication_close_perc_2019],
    'Telecommunication 2020 Performance': [telecommunication_close_perc_2020],
    'Telecommunication 2021 Performance': [telecommunication_close_perc_2021],
    'Technology 2019 Performance': [technology_close_perc_2019],
    'Technology 2020 Performance': [technology_close_perc_2020],
    'Technology 2021 Performance': [technology_close_perc_2021],
}

In [162]:
percentage_return_dict

{'Airlines 2019 Performance': [12.54],
 'Airlines 2020 Performance': [-34.29],
 'Airlines 2021 Performance': [-3.38],
 'Automobiles 2019 Performance': [17.17],
 'Automobiles 2020 Performance': [276.07],
 'Automobiles 2021 Performance': [26.05],
 'Consumer Products 2019 Performance': [16.86],
 'Consumer Products 2020 Performance': [24.5],
 'Consumer Products 2021 Performance': [9.67],
 'Health Care 2019 Performance': [24.23],
 'Health Care 2020 Performance': [11.24],
 'Health Care 2021 Performance': [15.27],
 'Hotels Restaurants and Leisure 2019 Performance': [17.27],
 'Hotels Restaurants and Leisure 2020 Performance': [16.02],
 'Hotels Restaurants and Leisure 2021 Performance': [30.13],
 'Logistics and Transportation 2019 Performance': [-2.67],
 'Logistics and Transportation 2020 Performance': [44.95],
 'Logistics and Transportation 2021 Performance': [14.27],
 'Pharmaceuticals 2019 Performance': [32.86],
 'Pharmaceuticals 2020 Performance': [19.62],
 'Pharmaceuticals 2021 Performance'

In [169]:
# Convert to DataFrame
percentage_return_df = pd.DataFrame.from_dict(percentage_return_dict)
percentage_return_df = percentage_return_df.T
percentage_return_df = percentage_return_df.rename(columns={'0': 'Percent Change'})
percentage_return_df

,0
Airlines 2019 Performance,12.54
Airlines 2020 Performance,-34.29
Airlines 2021 Performance,-3.38
Automobiles 2019 Performance,17.17
Automobiles 2020 Performance,276.07
Automobiles 2021 Performance,26.05
Consumer Products 2019 Performance,16.86
Consumer Products 2020 Performance,24.50
Consumer Products 2021 Performance,9.67
Health Care 2019 Performance,24.23
